**Install + Imports**

In [ ]:
# Install dependencies
!pip install pandas sqlalchemy matplotlib scipy

# Imports
import io, os, sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sqlalchemy import create_engine
from datetime import datetime
from google.colab import files

def info(msg):
    print(f"[INFO] {msg}")


**Upload CSVs**

In [ ]:
# Upload both CSV files
uploaded = files.upload()
# Expecting:
# - rainfall_area-wt_India_1901-2015.csv
# - RS_Session_266_AU_2112_A.csv

rainfall_file = "rainfall_area-wt_India_1901-2015.csv"
agri_file = "RS_Session_266_AU_2112_A.csv"

info(f"Uploaded files: {list(uploaded.keys())}")


Saving rainfall_area-wt_India_1901-2015.csv to rainfall_area-wt_India_1901-2015 (2).csv
Saving RS_Session_266_AU_2112_A.csv to RS_Session_266_AU_2112_A (2).csv
[INFO] Uploaded files: ['rainfall_area-wt_India_1901-2015 (2).csv', 'RS_Session_266_AU_2112_A (2).csv']


**Load DataFrames**

In [ ]:
# Load CSVs
rain_df = pd.read_csv(rainfall_file)
agri_df = pd.read_csv(agri_file)

info(f"Rainfall rows: {len(rain_df)}, Agriculture rows: {len(agri_df)}")
rain_df.head(), agri_df.head()


[INFO] Rainfall rows: 115, Agriculture rows: 17


(  REGION  YEAR   JAN   FEB   MAR   APR   MAY    JUN    JUL    AUG    SEP  \
 0  INDIA  1901  34.7  37.7  18.0  39.3  50.8  113.4  242.2  272.9  124.4   
 1  INDIA  1902   7.4   4.3  19.0  43.5  48.3  108.8  284.0  199.7  201.5   
 2  INDIA  1903  17.0   8.3  31.3  17.1  59.5  118.3  297.0  270.4  199.1   
 3  INDIA  1904  14.4   9.6  31.8  33.1  72.4  164.8  261.0  206.4  129.6   
 4  INDIA  1905  25.3  20.9  42.7  33.7  55.7   93.3  252.8  200.8  178.4   
 
      OCT   NOV   DEC  ANNUAL  Jan-Feb  Mar-May  Jun-Sep  Oct-Dec  
 0   52.7  38.0   8.3  1032.3     72.4    108.1    752.8     99.0  
 1   61.5  27.9  24.4  1030.2     11.7    110.8    794.0    113.8  
 2  117.9  36.9  17.7  1190.5     25.3    107.9    884.8    172.5  
 3   69.0  11.2  16.3  1019.8     24.0    137.4    761.8     96.6  
 4   51.4   9.7  10.5   975.3     46.2    132.2    725.4     71.6  ,
   Sl. No.    Crop  Production (Lakh Tons) - 2023-24  \
 0       1    Rice                           1378.25   
 1       2   Wh

**Normalize the Columns**

In [ ]:
def normalize_agri_df(df):
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]

    # Try to detect core columns
    state_col = next((c for c in df.columns if 'state' in c and 'code' not in c), None)
    district_col = next((c for c in df.columns if 'district' in c), None)
    year_col = next((c for c in df.columns if 'year' in c), None)
    crop_col = next((c for c in df.columns if 'crop' in c or 'commodity' in c), None)
    prod_col = next((c for c in df.columns if 'production' in c or 'yield' in c or 'qty' in c), None)

    # Fill missing with NaN safely
    df['state'] = df[state_col] if state_col else np.nan
    df['district'] = df[district_col] if district_col else np.nan
    df['year'] = pd.to_numeric(df[year_col], errors='coerce') if year_col else np.nan
    df['crop'] = df[crop_col] if crop_col else np.nan
    df['production_tonnes'] = pd.to_numeric(df[prod_col], errors='coerce') if prod_col else np.nan

    info(f"Detected columns — state:{state_col}, district:{district_col}, year:{year_col}, crop:{crop_col}, production:{prod_col}")
    return df[['state', 'district', 'year', 'crop', 'production_tonnes']]


def normalize_rainfall_df(df):
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]

    state_col = next((c for c in df.columns if 'state' in c), None)
    district_col = next((c for c in df.columns if 'district' in c), None)
    year_col = next((c for c in df.columns if 'year' in c), None)
    month_col = next((c for c in df.columns if 'month' in c), None)
    rain_col = next((c for c in df.columns if 'rain' in c or 'precip' in c), None)

    df['state'] = df[state_col] if state_col else np.nan
    df['district'] = df[district_col] if district_col else np.nan
    df['year'] = pd.to_numeric(df[year_col], errors='coerce') if year_col else np.nan
    df['month'] = df[month_col] if month_col else np.nan
    df['rainfall_mm'] = pd.to_numeric(df[rain_col], errors='coerce') if rain_col else np.nan

    info(f"Detected columns — state:{state_col}, district:{district_col}, year:{year_col}, month:{month_col}, rainfall:{rain_col}")
    return df[['state', 'district', 'year', 'month', 'rainfall_mm']]



**Normalize and Save to SQLite**

In [ ]:
agri_norm = normalize_agri_df(agri_df)
rain_norm = normalize_rainfall_df(rain_df)

DBPATH = "samarth_demo.db"
if os.path.exists(DBPATH):
    os.remove(DBPATH)
engine = create_engine(f"sqlite:///{DBPATH}", echo=False)
agri_norm.to_sql("production", engine, index=False)
rain_norm.to_sql("rainfall", engine, index=False)
info("Normalized tables saved into SQLite database ✅")


[INFO] Detected columns — state:None, district:None, year:None, crop:crop, production:production (lakh tons) - 2023-24
[INFO] Detected columns — state:None, district:None, year:year, month:None, rainfall:None
[INFO] Normalized tables saved into SQLite database ✅


**Compare Rainfall and Crops**

In [ ]:
def task_compare_rain_and_top_crops(state_x, state_y, N, crop_type, M=5):
    con = engine.connect()
    q_years = """
    SELECT year FROM rainfall
    WHERE state IN (:sx, :sy) AND year IS NOT NULL
    GROUP BY year
    HAVING COUNT(DISTINCT state) = 2
    ORDER BY year DESC LIMIT :n
    """
    years = [int(r[0]) for r in con.execute(q_years, {"sx":state_x, "sy":state_y, "n":N}).fetchall()]
    if not years:
        raise RuntimeError("No overlapping rainfall data found")

    q_rain = f"""
    SELECT state, AVG(rainfall_mm) AS avg_annual_mm
    FROM rainfall
    WHERE state IN (:sx, :sy) AND year IN ({','.join(map(str, years))})
    GROUP BY state
    """
    df_rain = pd.read_sql_query(q_rain, con, params={"sx":state_x, "sy":state_y})

    q_crop = f"""
    SELECT state, crop, SUM(production_tonnes) as total_tonnes
    FROM production
    WHERE state IN (:sx, :sy) AND year IN ({','.join(map(str, years))})
      AND crop LIKE :crop
    GROUP BY state, crop
    ORDER BY total_tonnes DESC
    """
    df_crop = pd.read_sql_query(q_crop, con, params={"sx":state_x,"sy":state_y,"crop":f"%{crop_type}%"})

    top_crops = {st: df_crop[df_crop["state"]==st].nlargest(M,"total_tonnes") for st in [state_x,state_y]}
    return df_rain, top_crops


**Run the Analysis**

In [ ]:
try:
    df_rain, top_crops = task_compare_rain_and_top_crops("Bihar", "Odisha", 5, "Rice", 3)
    print("Average Rainfall (last 5 overlapping years):")
    display(df_rain)
    print("\nTop Crops:")
    for st, df in top_crops.items():
        print(f"\n{st}:")
        display(df)
except Exception as e:
    print("Error:", e)


Error: Not an executable object: '\n    SELECT year FROM rainfall\n    WHERE state IN (:sx, :sy) AND year IS NOT NULL\n    GROUP BY year\n    HAVING COUNT(DISTINCT state) = 2\n    ORDER BY year DESC LIMIT :n\n    '


**Optional Plot**

In [ ]:
def plot_prod_and_rain(df_prod, df_rain, state):
    merged = pd.merge(df_prod, df_rain, on="year", how="inner")
    fig, ax1 = plt.subplots(figsize=(8,4))
    ax1.plot(merged["year"], merged["production_tonnes"], "o-", label="Production (tonnes)")
    ax2 = ax1.twinx()
    ax2.plot(merged["year"], merged["rainfall_mm"], "s--", color="orange", label="Rainfall (mm)")
    ax1.set_xlabel("Year")
    ax1.set_ylabel("Production")
    ax2.set_ylabel("Rainfall")
    plt.title(f"Rainfall vs Production — {state}")
    plt.tight_layout()
    plt.show()


In [ ]:
import sqlite3

conn = sqlite3.connect("samarth_demo.db")
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
conn.close()

print("Tables found in database:")
for t in tables:
    print(" -", t[0])


Tables found in database:
 - production
 - rainfall


In [ ]:
conn = sqlite3.connect("samarth_demo.db")

# Replace these with your real table names
agri_df = pd.read_sql("SELECT * FROM production", conn)
rain_df = pd.read_sql("SELECT * FROM rainfall", conn)

conn.close()


In [ ]:
from joblib import dump
data_dict = {"agriculture": agri_df, "rainfall": rain_df}
dump(data_dict, "samarth_data.joblib")
print("✅ Created samarth_data.joblib successfully")


✅ Created samarth_data.joblib successfully


In [ ]:
import pandas as pd

agri_df = pd.read_csv("RS_Session_266_AU_2112_A.csv")
print(agri_df.head())
print("\nColumns:\n", agri_df.columns)


  Sl. No.    Crop  Production (Lakh Tons) - 2023-24  \
0       1    Rice                           1378.25   
1       2   Wheat                           1132.92   
2       3   Maize                            376.65   
3       4  Barley                             16.99   
4       5   Jowar                             47.37   

   Productivity (Yield in kg/ha) - 2023-24  
0                                     2882  
1                                     3559  
2                                     3351  
3                                     3082  
4                                     1162  

Columns:
 Index(['Sl. No.', 'Crop', 'Production (Lakh Tons) - 2023-24',
       'Productivity (Yield in kg/ha) - 2023-24'],
      dtype='object')
